<a href="https://colab.research.google.com/github/Gzim30/Gzim30/blob/main/banks_project_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import sqlite3
import datetime

In [2]:
from google.colab import files
uploaded = files.upload()



Saving exchange_rate.csv to exchange_rate.csv


In [3]:
import pandas as pd

# Load the exchange rate CSV file into a DataFrame
exchange_rate_df = pd.read_csv('exchange_rate.csv')

# Display the DataFrame to verify that it was loaded correctly
print(exchange_rate_df)


  Currency   Rate
0      EUR   0.93
1      GBP   0.80
2      INR  82.95


In [6]:
# Code for ETL operations on Country-GDP data
# Importing the required libraries
import pandas as pd

def log_progress(message):
    ''' This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing'''

def extract(url, table_attribs):
    ''' This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing. '''
    return df

def transform(df, csv_path):
    ''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies'''
    return df

def load_to_csv(df, output_path):
    ''' This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.'''

def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final data frame to a database
    table with the provided name. Function returns nothing.'''

def run_query(query_statement, sql_connection):
    ''' This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. '''


In [7]:
import datetime

def log_progress(message):
    '''
    This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing.
    '''
    # Generate timestamp
    timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    # Log message format
    log_message = f"{timestamp} : {message}\n"

    # Write log message to file
    with open('code_log.txt', 'a') as log_file:
        log_file.write(log_message)

# Log the first message
log_progress("Preliminaries complete. Initiating ETL process")


In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

def log_progress(message):
    '''
    This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing.
    '''
    # Generate timestamp
    timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    # Log message format
    log_message = f"{timestamp} : {message}\n"

    # Write log message to file
    with open('code_log.txt', 'a') as log_file:
        log_file.write(log_message)

def extract(url):
    '''
    This function retrieves the required table data from the given URL
    and returns it as a Pandas DataFrame.
    '''
    try:
        # Send HTTP request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors

        # Parse HTML content
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the table with the given class
        table = soup.find('table', class_='wikitable')

        # Extract table data
        data = []
        for row in table.find_all('tr')[1:]:  # Exclude header row
            cols = row.find_all(['th', 'td'])
            cols = [col.text.strip() for col in cols]
            data.append(cols)

        # Convert data to DataFrame
        df = pd.DataFrame(data, columns=['Rank', 'Bank Name', 'Market Cap (US$ Billion)'])

        # Remove '\n' character from Market Cap column and convert to float
        df['Market Cap (US$ Billion)'] = df['Market Cap (US$ Billion)'].str.replace('\n', '').astype(float)

        return df

    except Exception as e:
        print(f"Error occurred: {e}")
        return None

# Function call for extract() and print the returning DataFrame
extracted_data = extract("https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks")
print(extracted_data)

# Log the completion of data extraction
log_progress("Data extraction complete. Initiating Transformation process")





  Rank                                Bank Name  Market Cap (US$ Billion)
0    1                           JPMorgan Chase                    432.92
1    2                          Bank of America                    231.52
2    3  Industrial and Commercial Bank of China                    194.56
3    4               Agricultural Bank of China                    160.68
4    5                                HDFC Bank                    157.91
5    6                              Wells Fargo                    155.87
6    7                        HSBC Holdings PLC                    148.90
7    8                           Morgan Stanley                    140.83
8    9                  China Construction Bank                    139.82
9   10                            Bank of China                    136.81


In [13]:
import numpy as np

def transform(df, exchange_rate_csv):
    # Read exchange rate CSV file and convert contents to a dictionary
    exchange_rate_df = pd.read_csv(exchange_rate_csv)
    exchange_rate_dict = dict(zip(exchange_rate_df['Currency'], exchange_rate_df['Rate']))

    # Add columns for MC_GBP_Billion, MC_EUR_Billion, and MC_INR_Billion
    df['MC_GBP_Billion'] = np.round(df['Market Cap (US$ Billion)'] * exchange_rate_dict['GBP'], 2)
    df['MC_EUR_Billion'] = np.round(df['Market Cap (US$ Billion)'] * exchange_rate_dict['EUR'], 2)
    df['MC_INR_Billion'] = np.round(df['Market Cap (US$ Billion)'] * exchange_rate_dict['INR'], 2)

    return df

# Function call for transform()
transformed_data = transform(extracted_data, 'exchange_rate.csv')
print(transformed_data)


  Rank                                Bank Name  Market Cap (US$ Billion)  \
0    1                           JPMorgan Chase                    432.92   
1    2                          Bank of America                    231.52   
2    3  Industrial and Commercial Bank of China                    194.56   
3    4               Agricultural Bank of China                    160.68   
4    5                                HDFC Bank                    157.91   
5    6                              Wells Fargo                    155.87   
6    7                        HSBC Holdings PLC                    148.90   
7    8                           Morgan Stanley                    140.83   
8    9                  China Construction Bank                    139.82   
9   10                            Bank of China                    136.81   

   MC_GBP_Billion  MC_EUR_Billion  MC_INR_Billion  
0          346.34          402.62        35910.71  
1          185.22          215.31        19204.5

In [14]:
def load_to_csv(df, output_path):
    '''
    This function saves the final data frame as a CSV file in the provided path.
    Function returns nothing.
    '''
    df.to_csv(output_path, index=False)
    print("Data saved to CSV file successfully.")

# Define the output path where you want to save the CSV file
output_path = '/content/banks_data_transformed.csv'

# Function call to save the transformed DataFrame to a CSV file
load_to_csv(transformed_data, output_path)


Data saved to CSV file successfully.


In [18]:
from google.colab import files

# Provide the correct path to the CSV file
csv_file_path = '/content/banks_data_transformed.csv'

# Download the CSV file
files.download(csv_file_path)




<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
import sqlite3

def run_queries(query, conn):
    """
    Executes the provided SQLite query and prints the query statement along with the query output.

    Parameters:
    - query: A string representing the SQLite query statement.
    - conn: SQLite3 Connection object.

    Returns:
    - None
    """
    try:
        # Create a cursor object
        cursor = conn.cursor()

        # Execute the query
        cursor.execute(query)

        # Fetch all results
        results = cursor.fetchall()

        # Print the query statement
        print("Query Statement:")
        print(query)

        # Print the query output
        print("Query Output:")
        for row in results:
            print(row)

    except sqlite3.Error as e:
        print("Error executing SQLite query:", e)

# Connect to an SQLite database
conn = sqlite3.connect(':memory:')  # In-memory database for demonstration purposes

# Create a table 'Largest_banks' for demonstration
create_table_query = """
CREATE TABLE IF NOT EXISTS Largest_banks (
    Name TEXT,
    MC_GBP_Billion REAL
);
"""
conn.execute(create_table_query)

# Insert some sample data into 'Largest_banks'
insert_data_query = """
INSERT INTO Largest_banks (Name, MC_GBP_Billion)
VALUES ('Bank A', 100),
       ('Bank B', 200),
       ('Bank C', 300),
       ('Bank D', 400),
       ('Bank E', 500);
"""
conn.execute(insert_data_query)

# Run the provided queries
queries = [
    "SELECT * FROM Largest_banks",
    "SELECT AVG(MC_GBP_Billion) FROM Largest_banks",
    "SELECT Name FROM Largest_banks LIMIT 5"
]

for query in queries:
    print("\nExecuting query:\n", query)
    run_queries(query, conn)

conn.close()



Executing query:
 SELECT * FROM Largest_banks
Query Statement:
SELECT * FROM Largest_banks
Query Output:
('Bank A', 100.0)
('Bank B', 200.0)
('Bank C', 300.0)
('Bank D', 400.0)
('Bank E', 500.0)

Executing query:
 SELECT AVG(MC_GBP_Billion) FROM Largest_banks
Query Statement:
SELECT AVG(MC_GBP_Billion) FROM Largest_banks
Query Output:
(300.0,)

Executing query:
 SELECT Name FROM Largest_banks LIMIT 5
Query Statement:
SELECT Name FROM Largest_banks LIMIT 5
Query Output:
('Bank A',)
('Bank B',)
('Bank C',)
('Bank D',)
('Bank E',)
